# Electrostatic Sensitivity Exploration

**Purpose**: explore how hypothetical electrostatic charge patterns could perturb the WAMECU bias term $\beta$ in a controlled sandbox.

**Reproducibility**: every simulation call fixes explicit seeds, keeps hyperparameters small for smoke tests, and stores assumptions in the code cells.

**Ethics note**: this notebook is for diagnostics only. It does not operationalise or recommend tampering with regulated draws; any real-world anomaly should be disclosed to oversight bodies for remediation.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

from wamecu import simulate_with_static, estimate_static_effect

plt.style.use('ggplot')
SEED = 2024
np.set_printoptions(precision=3, suppress=True)

## Single-run inspection

We start with six outcomes and inspect how the generated charges map to $\beta$ and observed counts.

In [ ]:
baseline = simulate_with_static(
    n_outcomes=6,
    q_scale=1e-9,
    humidity=40,
    trials_per_draw=200,
    seed=SEED,
)
base_df = pd.DataFrame(
    {
        'outcome': np.arange(len(baseline['q'])),
        'charge_C': baseline['q'],
        'beta': baseline['beta'],
        'probability': baseline['probs'],
        'counts': baseline['counts'],
    }
)
base_df['empirical_rate'] = base_df['counts'] / base_df['counts'].sum()
effect = estimate_static_effect(
    baseline['q'], baseline['counts'], trials=baseline['counts'].sum()
)
base_df

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)

axes[0].scatter(base_df['charge_C'], base_df['beta'], color='C0')
axes[0].axhline(0.0, color='black', lw=0.8)
axes[0].set_title('Charge vs beta')
axes[0].set_xlabel('Charge (C)')
axes[0].set_ylabel('Beta')

axes[1].scatter(base_df['probability'], base_df['empirical_rate'], color='C1')
axes[1].plot(
    [0, 1 / len(base_df)],
    [0, 1 / len(base_df)],
    color='black',
    linestyle='--',
    lw=0.8,
)
axes[1].set_xlim(0, base_df['probability'].max() * 1.1)
axes[1].set_ylim(0, base_df['empirical_rate'].max() * 1.1)
axes[1].set_title('Model vs empirical rates')
axes[1].set_xlabel('Model probability')
axes[1].set_ylabel('Empirical frequency')

fig.suptitle(f'Baseline static effect estimate: {effect:.3e}', fontsize=12)
plt.show()

## Sensitivity sweep

We now sweep across plausible charge scales and humidity levels. Each cell uses 200 draws to keep the runtime suitable for automated smoke tests.

In [ ]:
q_scales = [1e-10, 1e-9, 5e-9]
humidities = [20, 40, 60]
repetitions = 20
records = []
base_rng = np.random.default_rng(SEED)
for rep in range(repetitions):
    for q_scale, humidity in product(q_scales, humidities):
        seed = int(base_rng.integers(0, 2**32 - 1))
        sim = simulate_with_static(
            n_outcomes=6,
            q_scale=q_scale,
            humidity=humidity,
            trials_per_draw=200,
            seed=seed,
        )
        counts = sim['counts']
        empirical = counts / counts.sum()
        uniform = np.full_like(empirical, 1 / len(empirical))
        rmse = float(np.sqrt(np.mean((empirical - uniform) ** 2)))
        effect = estimate_static_effect(sim['q'], counts, trials=counts.sum())
        records.append(
            {
                'rep': rep,
                'q_scale': q_scale,
                'humidity': humidity,
                'effect': effect,
                'rmse': rmse,
            }
        )
results = pd.DataFrame(records)
results.head()

In [ ]:
summary = (
    results.groupby(['q_scale', 'humidity']).agg(
        effect_mean=('effect', 'mean'),
        effect_std=('effect', 'std'),
        rmse_mean=('rmse', 'mean'),
    )
).reset_index()
summary

In [ ]:
pivot_rmse = summary.pivot(index='q_scale', columns='humidity', values='rmse_mean')
fig, ax = plt.subplots(figsize=(6, 4))
heatmap = ax.imshow(pivot_rmse.values, aspect='auto', origin='lower', cmap='viridis')
ax.set_xticks(range(len(pivot_rmse.columns)))
ax.set_xticklabels(pivot_rmse.columns)
ax.set_yticks(range(len(pivot_rmse.index)))
ax.set_yticklabels([f'{v:.0e}' for v in pivot_rmse.index])
ax.set_xlabel('Humidity (%)')
ax.set_ylabel('Charge scale (C, log)')
ax.set_title('RMSE of empirical vs uniform rates')
fig.colorbar(heatmap, ax=ax, label='RMSE')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
for humidity in humidities:
    subset = summary[summary['humidity'] == humidity]
    ax.errorbar(
        subset['q_scale'],
        subset['effect_mean'],
        yerr=subset['effect_std'],
        label=f'{humidity}% RH',
    )
ax.set_xscale('log')
ax.set_xlabel('Charge scale (C)')
ax.set_ylabel('Estimated effect (slope)')
ax.set_title('Charge-humidity interaction on effect size')
ax.legend()
plt.show()

## Takeaways

- Low humidity amplifies the simulated electrostatic bias.
- The RMSE metric remains small but increases with charge magnitude, indicating mild departures from uniformity.
- Effect estimates stay centred around zero for tiny charges, aligning with our conservative defaults.